<a href="https://colab.research.google.com/github/developerabhi14/ML-Notebooks/blob/main/FineTuning_BERT_for_Toxic_Comment_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
! pip install transformers

In [5]:
import pandas as pd

data=pd.read_csv("train.csv",encoding="utf-8")
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [6]:
data['toxic'].value_counts()

,count
toxic,
0,144277
1,15294


In [7]:
data=data[['comment_text', 'toxic']]
data=data[0:100]
data.head()

,comment_text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

In [9]:
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')
model=BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
model=model.to('cpu')
sample_data=['I am eating apple', 'I am playing cricket']
tokenizer(sample_data, padding=True, truncation=True, max_length=512)

{'input_ids': [[101, 1045, 2572, 5983, 6207, 102], [101, 1045, 2572, 2652, 4533, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [12]:
X=list(data['comment_text'])
y=list(data['toxic'])

X_train, X_val, y_train, y_val=train_test_split(X,y, test_size=0.2, stratify=y)
X_train_tokenized=tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized=tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [13]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [14]:
print(X_train_tokenized['input_ids'][0])

[101, 1000, 4586, 10258, 20060, 2024, 2025, 2467, 23476, 999, 2104, 10988, 2009, 2003, 3090, 2008, 1000, 1000, 1037, 4586, 10258, 13808, 2467, 2038, 2416, 19490, 2608, 1012, 1000, 1000, 2023, 23617, 2003, 3432, 2025, 2995, 999, 2429, 2000, 8856, 5622, 10322, 28109, 1010, 1000, 1000, 1996, 2738, 14477, 4779, 26884, 12052, 14438, 2024, 2011, 2521, 1996, 2087, 2691, 3528, 1012, 1000, 1000, 8299, 1024, 1013, 1013, 7479, 1012, 2049, 1012, 10250, 15007, 1012, 3968, 2226, 1013, 1066, 9593, 1013, 4586, 26775, 27268, 9777, 1013, 17218, 1013, 17218, 1012, 1044, 21246, 1001, 15401, 2619, 2428, 2342, 2000, 2202, 1037, 2298, 2012, 2010, 2609, 1998, 2131, 8866, 2125, 1997, 2009, 2138, 1045, 2145, 2156, 1037, 11519, 2193, 1997, 6904, 4877, 6447, 2006, 2023, 3931, 1012, 1006, 9641, 2033, 10047, 2047, 2012, 2023, 1998, 2123, 2102, 2215, 2000, 10086, 2505, 1007, 1000, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [15]:
len(X_train), len(X_val)

(80, 20)

In [16]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels=None):
    self.encodings=encodings
    self.labels=labels

  def __getitem__(self, idx):
    item={key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    if self.labels:
      item['labels']=torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.encodings['input_ids'])

In [17]:
train_dataset=Dataset(X_train_tokenized, y_train)
val_dataset=Dataset(X_val_tokenized, y_val)

In [18]:
train_dataset[5]

{'input_ids': tensor([  101,  4037,  4931,  2035,  1010,  1045,  2001,  3241,  1997,  2893,
          2870,  1037,  4037,  2000,  4653,  2026,  4620,  1998,  2023,  2001,
          1996, 10036,  4355,  2518,  1045,  2071,  2424,  1012,  1045,  2123,
          1005,  1056,  2113,  2055,  2500,  1010,  2021, 21554,  1998, 29454,
         13355,  1010,  2017,  4364,  2031, 11744,  1010,  2079,  2017,  2228,
          2009,  2003,  4190,  4183,  1029,  2071,  2017,  3073,  2151,  2488,
         15955,  1029,  2393,  2013,  2151,  2060,  9054,  2266,  2003,  2036,
         12315,  1012,  4283,  1006,  2831,  1007,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [19]:
def compute_metrics(p):
  print(type(p))
  pred,labels=p
  pred=np.argmax(pred, axis=1)

  accuracy=accuracy_score(y_true=labels, y_pred=pred)
  recall=recall_score(y_true=labels, y_pred=pred)
  precision=precision_score(y_true=labels, y_pred=pred)
  f1=f1_score(y_true=labels, y_pred=pred)

  return {"accuracy":accuracy, "precision":precision, "recall":recall, "f1":f1}

In [22]:
# Define Trainer

args=TrainingArguments(
    output_dir='output',
    num_train_epochs=5,
    fp16=True, # Enable FP16 Traininng
    per_device_train_batch_size=8
)

trainer=Trainer(
    model=model, args=args,train_dataset=train_dataset, eval_dataset=val_dataset, compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: avskh to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


In [ ]:
trainer.evaluate()

In [ ]:
text="that was a good point"
input=tokenizer(text, padding=True, truncation=True, return_tensors='pt').to('cuda')
outputs=model(**input)
predictions=torch.nn.functional.softmax(outputs.logits,dim=-1)
predictions=predictions.cpu().detach().numpy()
predictions

In [ ]:
trainer.save_model('CustomModel')

In [ ]:
model_2=BertForSequenceClassification.from_pretrained('CustomModel')
model_2.to('cuda')

In [ ]:
text="you rock"
input=tokenizer(text, padding=True, truncation=True, return_tensors='pt').to('cuda')
outputs=model_2(**input)
predictions=torch.nn.functional.softmax(outputs.logits,dim=-1)
predictions=predictions.cpu().detach().numpy()
predictions

# Benchmarking inference speed of the model before eval and after quantization optimization


In [ ]:
import torch
import time
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load model and tokenizer
model_name = "CustomModel"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load FP32 (default precision) model
model_fp32 = AutoModelForSequenceClassification.from_pretrained(model_name)
model_fp32.eval()

# Convert to FP16
model_fp16 = AutoModelForSequenceClassification.from_pretrained(model_name)
model_fp16.half().eval()

# Move to GPU (if available)
device = "cuda" if torch.cuda.is_available() else "cpu"
model_fp32.to(device)
model_fp16.to(device)

# Sample input
text = "This is a benchmark test."
inputs = tokenizer(text, return_tensors="pt").to(device)

# Benchmark function
def benchmark(model, inputs, num_runs=100):
    times = []
    with torch.no_grad():
        for _ in range(num_runs):
            start_time = time.time()
            _ = model(**inputs)
            times.append(time.time() - start_time)
    return sum(times) / num_runs  # Average latency

# Measure latency
fp32_time = benchmark(model_fp32, inputs)
fp16_time = benchmark(model_fp16, inputs)

print(f"FP32 Avg Inference Time: {fp32_time:.6f} sec")
print(f"FP16 Avg Inference Time: {fp16_time:.6f} sec")
print(f"Speedup: {fp32_time / fp16_time:.2f}x")
